In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import matplotlib.pyplot as plt

# Try imports for explainability
HAS_SHAP = True
HAS_LIME = True
try:
    import shap
except Exception:
    HAS_SHAP = False

try:
    from lime import lime_tabular
except Exception:
    HAS_LIME = False

# ---------------------------
# Config / file
# ---------------------------
CSV = "20231225_dfall_obs_data_and_spectral_features_revision1_n469.csv"
if not os.path.exists(CSV):
    raise FileNotFoundError(f"Place '{CSV}' in the same folder as this script and run it.")

# ---------------------------
# Load data (same as your original)
# ---------------------------
df = pd.read_csv(CSV)

if "Context2" not in df.columns:
    raise ValueError("Expected column 'Context2' not found in dataset!")

target_col = "Context2"
# encode target labels to numeric
df[target_col] = df[target_col].astype("category").cat.codes

# keep numeric features only
X = df.select_dtypes(include=[np.number]).copy()
y = df[target_col].copy()
X = X.drop(columns=[target_col], errors='ignore')

# simple missing handling (same as original)
X = X.fillna(X.mean())

# train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# ---------------------------
# pipeline components (same idea as yours)
# ---------------------------
scaler = StandardScaler()
selector = SelectKBest(score_func=f_classif, k=min(60, X_train.shape[1]))
pca = PCA(n_components=min(25, X_train.shape[1]), random_state=42)
imputer = SimpleImputer(strategy="mean")  # defensive

# ---------------------------
# models (same hyperparams as you)
# ---------------------------
models = {
    "Random Forest": RandomForestClassifier(
        n_estimators=400, max_depth=25, min_samples_split=3, min_samples_leaf=1, random_state=42, n_jobs=-1
    ),
    "XGBoost": XGBClassifier(
        n_estimators=400, learning_rate=0.04, max_depth=8, subsample=0.85, colsample_bytree=0.85, random_state=42,
        use_label_encoder=False, eval_metric='mlogloss', n_jobs=1
    ),
    "SVM": SVC(C=2.5, kernel='rbf', gamma='auto', probability=True, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(
        n_estimators=300, learning_rate=0.06, max_depth=6, random_state=42
    ),
    "Logistic Regression": LogisticRegression(
        C=3.0, max_iter=2000, solver='lbfgs', random_state=42
    )
}

# stacking ensemble
estimators = [
    ('rf', models['Random Forest']),
    ('xgb', models['XGBoost']),
    ('svm', models['SVM'])
]

stacked = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(max_iter=1500, random_state=42),
    cv=5,
    n_jobs=-1
)

# ---------------------------
# Train & evaluate original pipelines (with selector + PCA)
# store fitted pipelines for later extraction
# ---------------------------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("=== TRAINING ORIGINAL MODELS (with SelectKBest + PCA) ===")
fitted_pipes = {}
for name, model in {**models, "Stacked Ensemble": stacked}.items():
    pipe = Pipeline([
        ('imputer', imputer),
        ('scaler', scaler),
        ('selector', selector),
        ('pca', pca),
        ('model', model)
    ])
    # fit
    pipe.fit(X_train, y_train)
    # store
    fitted_pipes[name] = pipe
    # evaluate
    preds = pipe.predict(X_test)
    acc = accuracy_score(y_test, preds)
    try:
        scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
    except Exception:
        scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=1)
    print(f"{name:20s} - Acc: {acc:.3f}, CV: {scores.mean():.3f} ± {scores.std():.3f}")

print("\nDone training original pipelines.\n")

# ---------------------------
# EXPLAINABILITY PREP
# Reasoning:
# - PCA components are not human-friendly.
# - For SHAP/LIME we will explain models trained on the selected features (after SelectKBest but BEFORE PCA).
# - So: reuse the fitted 'selector' (it contains which features were selected) to create explainable datasets.
# ---------------------------

# get the fitted selector from any of the pipelines (they all used same 'selector' instance type)
# but pipeline created separate SelectKBest objects for each model, so we should use the one from any fitted pipe
# choose Random Forest pipeline's selector (exists because we fitted it above)
rf_pipe = fitted_pipes.get("Random Forest")
if rf_pipe is None:
    raise RuntimeError("Random Forest pipeline not found in fitted pipelines (unexpected).")

fitted_selector = rf_pipe.named_steps['selector']
# we also need the scaler used before selector
fitted_scaler = rf_pipe.named_steps['scaler']
fitted_imputer = rf_pipe.named_steps['imputer']

# transform train/test up to selector (no PCA)
X_train_imputed = pd.DataFrame(fitted_imputer.transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_imputed  = pd.DataFrame(fitted_imputer.transform(X_test),  columns=X_test.columns,  index=X_test.index)

# scale then select
X_train_scaled = pd.DataFrame(fitted_scaler.transform(X_train_imputed), columns=X_train.columns, index=X_train.index)
X_test_scaled  = pd.DataFrame(fitted_scaler.transform(X_test_imputed),  columns=X_test.columns,  index=X_test.index)

# selected feature mask & names
mask = fitted_selector.get_support()
selected_feature_names = X_train.columns[mask].tolist()
print("Selected feature count for explanations:", len(selected_feature_names))
print("Selected features (first 20):", selected_feature_names[:20])

# datasets for explainable models (selected features only)
X_train_selected = pd.DataFrame(fitted_selector.transform(X_train_scaled), columns=selected_feature_names, index=X_train.index)
X_test_selected  = pd.DataFrame(fitted_selector.transform(X_test_scaled),  columns=selected_feature_names, index=X_test.index)

# ---------------------------
# Train interpretable models (no PCA) to explain with SHAP
# We'll train a RandomForest and XGBoost on selected features.
# ---------------------------
print("\nTraining explainable RandomForest and XGBoost on selected features (no PCA) ...")
expl_rf = RandomForestClassifier(n_estimators=300, max_depth=20, random_state=42, n_jobs=-1)
expl_xgb = XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=6, subsample=0.85,
                        colsample_bytree=0.85, random_state=42, use_label_encoder=False, eval_metric='mlogloss', n_jobs=1)

expl_rf.fit(X_train_selected, y_train)
expl_xgb.fit(X_train_selected, y_train)

# quick test performance of explainable models (so you compare)
print("\nExplainable model performance on hold-out test (selected features):")
for name, m, Xs in [("RF-explain", expl_rf, X_test_selected), ("XGB-explain", expl_xgb, X_test_selected)]:
    preds = m.predict(Xs)
    acc = accuracy_score(y_test, preds)
    f1w = f1_score(y_test, preds, average='weighted')
    print(f"{name:12s} - Acc: {acc:.3f}, F1w: {f1w:.3f}")

# ---------------------------
# SHAP explanations (if available)
# ---------------------------
if HAS_SHAP:
    print("\nRunning SHAP explanations (this may take a bit)...")
    # TreeExplainer for tree models
    try:
        # RandomForest SHAP
        explainer_rf = shap.TreeExplainer(expl_rf)
        shap_values_rf = explainer_rf.shap_values(X_test_selected)  # list per class for multiclass

        # save summary plot (class-agnostic mean(|shap|) across classes)
        plt.figure(figsize=(8,6))
        # For multiclass, shap.summary_plot expects shap_values as array-like or list; to get global importance we compute mean abs
        if isinstance(shap_values_rf, list):
            # compute mean(|shap|) across classes
            mean_abs = np.mean([np.abs(sv) for sv in shap_values_rf], axis=0)
            # use shap's bar_plot via pandas
            importances = np.mean(mean_abs, axis=0)
        else:
            importances = np.mean(np.abs(shap_values_rf), axis=0)

        imp_df = pd.Series(importances, index=selected_feature_names).sort_values(ascending=False)
        topn = min(30, len(imp_df))
        imp_df.head(topn).plot.barh(figsize=(8, max(6, topn*0.25)))
        plt.gca().invert_yaxis()
        plt.title("RF (selected features) - SHAP mean |value| importance (top {})".format(topn))
        plt.tight_layout()
        plt.savefig("shap_rf_feature_importance.png", dpi=150)
        plt.close()
        print("Saved shap_rf_feature_importance.png")

        # Also produce SHAP summary_plot (requires shap plotting; will save)
        try:
            shap.summary_plot(shap_values_rf, X_test_selected, show=False, plot_size=(8,6))
            plt.savefig("shap_rf_summary_plot.png", dpi=150, bbox_inches='tight')
            plt.close()
            print("Saved shap_rf_summary_plot.png")
        except Exception as e:
            print("Could not create shap.summary_plot (display backend). Skipping. Err:", e)

        # Print top 10 features by mean absolute shap across classes
        top_features = imp_df.head(10)
        print("\nTop 10 features by SHAP (RF):")
        print(top_features)

        # Example: per-class top features (if multiclass)
        if isinstance(shap_values_rf, list):
            for class_idx, sv in enumerate(shap_values_rf):
                cls_importances = pd.Series(np.mean(np.abs(sv), axis=0), index=selected_feature_names).sort_values(ascending=False)
                print(f"\nTop 8 features for class index {class_idx}:")
                print(cls_importances.head(8))
    except Exception as e:
        print("SHAP RF explanation failed:", e)

    # XGBoost SHAP
    try:
        explainer_xgb = shap.TreeExplainer(expl_xgb)
        shap_values_xgb = explainer_xgb.shap_values(X_test_selected)
        # global importance
        if isinstance(shap_values_xgb, list):
            mean_abs_xgb = np.mean([np.abs(sv) for sv in shap_values_xgb], axis=0)
            importances_xgb = np.mean(mean_abs_xgb, axis=0)
        else:
            importances_xgb = np.mean(np.abs(shap_values_xgb), axis=0)
        imp_df_xgb = pd.Series(importances_xgb, index=selected_feature_names).sort_values(ascending=False)
        topn = min(30, len(imp_df_xgb))
        plt.figure(figsize=(8, max(3, topn*0.25)))
        imp_df_xgb.head(topn).plot.barh()
        plt.gca().invert_yaxis()
        plt.title("XGB (selected features) - SHAP mean |value| importance")
        plt.tight_layout()
        plt.savefig("shap_xgb_feature_importance.png", dpi=150)
        plt.close()
        print("Saved shap_xgb_feature_importance.png")
        print("\nTop 10 features by SHAP (XGB):")
        print(imp_df_xgb.head(10))
    except Exception as e:
        print("SHAP XGB explanation failed:", e)

else:
    print("\nSHAP not installed. To enable SHAP explanations run: pip install shap")

# ---------------------------
# LIME explanation for one test instance (optional)
# ---------------------------
if HAS_LIME:
    print("\nRunning LIME explanation for one test instance (on selected features)...")
    # lime needs numpy arrays and feature names
    X_train_arr = X_train_selected.values
    feature_names = selected_feature_names
    class_names = [str(c) for c in sorted(y.unique())]

    lime_explainer = lime_tabular.LimeTabularExplainer(
        training_data=X_train_arr,
        feature_names=feature_names,
        class_names=class_names,
        mode='classification',
        discretize_continuous=True
    )

    # pick one test sample
    sample_idx = X_test_selected.index[0]
    sample = X_test_selected.loc[sample_idx].values

    # we need a predict_proba function for the explainer - use the explainable XGB model
    def predict_proba_fn(x):
        # x is 2D numpy array in selected-feature space
        return expl_xgb.predict_proba(x)

    exp = lime_explainer.explain_instance(sample, predict_proba_fn, num_features=min(15, len(feature_names)))
    html_path = "lime_explanation_sample0.html"
    exp.save_to_file(html_path)
    print("Saved LIME explanation to", html_path)

else:
    print("\nLIME not installed. To enable LIME run: pip install lime")

=== TRAINING ORIGINAL MODELS (with SelectKBest + PCA) ===


Traceback (most recent call last):
  File "/home/easwer/.local/share/mise/installs/python/3.13.7/lib/python3.13/multiprocessing/resource_tracker.py", line 295, in main
    raise ValueError(
        f'Cannot register {name} for automatic cleanup: '
        f'unknown resource type {rtype}')
ValueError: Cannot register /dev/shm/joblib_memmapping_folder_37219_bb99b034127c4c46bf726ad85755ee91_ae40cc019def497fbc8cbecbb1bef16f for automatic cleanup: unknown resource type folder
Traceback (most recent call last):
  File "/home/easwer/.local/share/mise/installs/python/3.13.7/lib/python3.13/multiprocessing/resource_tracker.py", line 295, in main
    raise ValueError(
        f'Cannot register {name} for automatic cleanup: '
        f'unknown resource type {rtype}')
ValueError: Cannot register /loky-37219-jzg79z5n for automatic cleanup: unknown resource type semlock
Traceback (most recent call last):
  File "/home/easwer/.local/share/mise/installs/python/3.13.7/lib/python3.13/multiprocessing/reso

Random Forest        - Acc: 0.638, CV: 0.536 ± 0.036


Traceback (most recent call last):
  File "/home/easwer/.local/share/mise/installs/python/3.13.7/lib/python3.13/multiprocessing/resource_tracker.py", line 295, in main
    raise ValueError(
        f'Cannot register {name} for automatic cleanup: '
        f'unknown resource type {rtype}')
ValueError: Cannot register /dev/shm/joblib_memmapping_folder_37219_57f6cc20917d48d8925d1c97941e3065_b607546453504418864876156f821f42 for automatic cleanup: unknown resource type folder
Traceback (most recent call last):
  File "/home/easwer/.local/share/mise/installs/python/3.13.7/lib/python3.13/multiprocessing/resource_tracker.py", line 295, in main
    raise ValueError(
        f'Cannot register {name} for automatic cleanup: '
        f'unknown resource type {rtype}')
ValueError: Cannot register /dev/shm/joblib_memmapping_folder_37219_bb99b034127c4c46bf726ad85755ee91_f27c01082efc44b29956a4bb113dab61 for automatic cleanup: unknown resource type folder


XGBoost              - Acc: 0.617, CV: 0.528 ± 0.052


...

Logistic Regression  - Acc: 0.777, CV: 0.565 ± 0.048


Traceback (most recent call last):
  File "/home/easwer/.local/share/mise/installs/python/3.13.7/lib/python3.13/multiprocessing/resource_tracker.py", line 295, in main
    raise ValueError(
        f'Cannot register {name} for automatic cleanup: '
        f'unknown resource type {rtype}')
ValueError: Cannot register /dev/shm/joblib_memmapping_folder_37219_bb99b034127c4c46bf726ad85755ee91_a7b3454754234047a56699b7be4f7147 for automatic cleanup: unknown resource type folder
Traceback (most recent call last):
  File "/home/easwer/.local/share/mise/installs/python/3.13.7/lib/python3.13/multiprocessing/resource_tracker.py", line 295, in main
    raise ValueError(
        f'Cannot register {name} for automatic cleanup: '
        f'unknown resource type {rtype}')
ValueError: Cannot register /dev/shm/joblib_memmapping_folder_37219_33660e51c2bf48689b5d298396fc834a_1fafc109b9274f86af33a4ecc56eb282 for automatic cleanup: unknown resource type folder
Traceback (most recent call last):
  File "/home

Stacked Ensemble     - Acc: 0.723, CV: 0.555 ± 0.054

Done training original pipelines.

Selected feature count for explanations: 60
Selected features (first 20): ['selec', 'Distance', 'Quality', 'CallerAge', 'Elicitor1Age', 'Cert_ID_Bin', 'sprsMed', 'sprsMbw', 'sprsEqbw', 'sprsMc', 'V1', 'V3', 'V4', 'V5', 'V6', 'V7', 'V9', 'V10', 'V11', 'V14']

Training explainable RandomForest and XGBoost on selected features (no PCA) ...

Explainable model performance on hold-out test (selected features):
RF-explain   - Acc: 0.840, F1w: 0.842
XGB-explain  - Acc: 0.819, F1w: 0.819

Running SHAP explanations (this may take a bit)...
SHAP RF explanation failed: Data must be 1-dimensional, got ndarray of shape (60, 3) instead
SHAP XGB explanation failed: Data must be 1-dimensional, got ndarray of shape (60, 3) instead

Running LIME explanation for one test instance (on selected features)...
Saved LIME explanation to lime_explanation_sample0.html


<Figure size 800x600 with 0 Axes>

Traceback (most recent call last):
  File "/home/easwer/.local/share/mise/installs/python/3.13.7/lib/python3.13/multiprocessing/resource_tracker.py", line 295, in main
    raise ValueError(
        f'Cannot register {name} for automatic cleanup: '
        f'unknown resource type {rtype}')
ValueError: Cannot register /loky-37219-86h75a5c for automatic cleanup: unknown resource type semlock
Traceback (most recent call last):
  File "/home/easwer/.local/share/mise/installs/python/3.13.7/lib/python3.13/multiprocessing/resource_tracker.py", line 295, in main
    raise ValueError(
        f'Cannot register {name} for automatic cleanup: '
        f'unknown resource type {rtype}')
ValueError: Cannot register /loky-37219-ps9noobi for automatic cleanup: unknown resource type semlock
Traceback (most recent call last):
  File "/home/easwer/.local/share/mise/installs/python/3.13.7/lib/python3.13/multiprocessing/resource_tracker.py", line 295, in main
    raise ValueError(
        f'Cannot register 